In [209]:
import pandas as pd
import numpy as np

In [210]:
df_c = pd.read_csv('../data/comments_data.csv')
df_p = pd.read_csv('../data/products_data.csv')
df_s = pd.read_csv('../data/swatches_data.csv')

#### User status
- A lot of additional classes that are common across all status
- Remove prefixes to the actual labels that we're interested in like 'Graphic' or 'Label'

In [36]:
df_c.status = df_c.status.str.replace('BVRRBadgeGraphic', '')
df_c.status = df_c.status.str.replace('BVRRBadgeLabel', '')
df_c.status = df_c.status.str.replace('BVRRReviewBadgeLabel', '')
df_c.status = df_c.status.str.replace('BVRRReviewBadgeGraphic', '')
df_c.status = df_c.status.str.replace('Graphic', '')
df_c.status = df_c.status.str.replace('Label', '')
df_c.status = df_c.status.str.replace('BVRR', '')

In [67]:
# delete null entries
df_p = df_p[df_p.product_id.notnull()]

#### String to int for count values

In [48]:
df_c.star_count = df_c.star_count.str.extract('(\d) out of', expand=True)

In [50]:
df_c.star_count = df_c.star_count.astype(int)

In [60]:
df_p.love_count = df_p.love_count.str.replace('K', '000')

In [70]:
df_p.love_count = df_p.love_count.astype(int)

/Users/josephinetirtanata/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [84]:
for p_id in df_p[df_p.rev_count.isnull()].product_id:
    rev_count = df_c[df_c.product_id == p_id].shape[0]
    df_p.loc[df_p.product_id==p_id, 'rev_count'] = rev_count

/Users/josephinetirtanata/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [91]:
df_p.rev_count = df_p.rev_count.astype(int)

/Users/josephinetirtanata/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


#### Splitting comment df to comment and user df

In [212]:
df_c.username = df_c.username.str.strip()

In [213]:
df_u = df_c[['username', 'age', 'eye_color', 'skin_tone', 'status']]

In [215]:
df_u = df_u[df_u.username != 'Anonymous']

In [216]:
df_info = df_u[['age', 'eye_color', 'skin_tone', 'status']].dropna(how='all')

In [217]:
df_info = df_info.join(df_u[['username']])

In [218]:
df_u = df_u.groupby('username').head(1)

In [219]:
df_u.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445419 entries, 0 to 848422
Data columns (total 5 columns):
username     445418 non-null object
age          273290 non-null object
eye_color    245266 non-null object
skin_tone    256534 non-null object
status       262933 non-null object
dtypes: object(5)
memory usage: 20.4+ MB


In [220]:
for col in ['age', 'eye_color', 'skin_tone', 'status']:
    df_temp = df_info[['username', col]].dropna().drop_duplicates('username').set_index('username')
    df_u = df_u.drop(col, axis=1).join(df_temp, on='username')

In [221]:
df_u.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445419 entries, 0 to 848422
Data columns (total 5 columns):
username     445418 non-null object
age          287457 non-null object
eye_color    264796 non-null object
skin_tone    274864 non-null object
status       277611 non-null object
dtypes: object(5)
memory usage: 20.4+ MB


#### Save 

In [223]:
df_u.to_csv('../data/users_data.csv', index=False)
df_c.to_csv('../data/comments_data.csv', index=False)

In [92]:
df_p.to_csv('../data/products_data.csv', index=False)
df_s.to_csv('../data/swatches_data.csv', index=False)